# Sobol' Sensitivity Analysis
### Sampling, Model execution, Index calculation and visualization

This notebook allow to run all relevant code to perform a Sobol' sensitivity analysis on the desert ant foraging model. The following steps are performed:

1. Configure model parameters and create samples
2. Run the model for each sample
3. Compute movement indices (in external R Script!)
4. Compute sobol indices
5. Visualize Sensitivity Analysis results (Sobol indices and model output variation)

In [ ]:
# load necessary packages
import agentpy as ap
import pandas
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# import the model
%run ./Model_Submission.ipynb

### Sensitivty analysis configuration
For configuration of the sensitivity analysis we need to set the _n_ for the saltelli sampling methods. _n_ should be a power of 2. General rule is, the higher _n_ the better the more accurate the results. BUT increasing _n_ comes at higher computational and storage cost! 
I suggest using _n_ = 512, which was used in the thesis. 
Computation took ~3 hours, using a Apple Macbook Pro with M1 Pro chip, 8-core CPU with 6 performance cores and 2 efficiency cores and 16GB unified memory. Model outputs used ~51GB of storage.

(Note: Using _n_ < 16 causes errors in sobol calculation or visualization. Choose _n_ > 16)

**Running the model with default configuration (n = 512, n_jobs=-1) may take several hours, requires 50+ GB free disk space and occupies all CPUs as well as a significant amount of RAM.**

In [ ]:
# configure n for saltelli sampling (value shall be power of two)
saltelli_n = 512

# set experiment id (default is current time)
experiment_id = int(time.time())

In [ ]:
parameters = {
    'agents': 10,
    'timestep': 1/50,
    'veg_speed_median': 0.11311, # mean of means ant 03 (4 tracks) & 11 (3 tracks)
    'veg_speed_sd': 0.09069,  # mean of sd ant 03 (4 tracks) & 11 (3 tracks)
    'open_speed_median': 0.19387, # mean of means ant 03 (4 tracks) & 11 (3 tracks)
    'open_speed_sd': 0.08409, # mean of sd ant 03 (4 tracks) & 11 (3 tracks)
    'speed_impact': ap.Range(0.25, 2),
    'veg_speed_impact': ap.Range(0.25, 2),
    'kphi': 0.12,
    'kphi_impact': ap.Range(0.5, 2),
    'sigma_impact': ap.Range(0.5, 2),
    'model_sigma2': 0.24,
    'nest': str(complex(0,0)),
    'context': ap.Values(False, True),
    'env_ant': 3,
    'foraging_runs': 3,
    'cookie_mean': 8,
    'cookie_sd': 0,
    'min_feeder_dist': 2,
    'pi_k': 0.68,
    'place_k': 7.5,
    'place_d': 4,
    'iphi_mean': 0.5*np.pi,
    'iphi_sd': 0.2,
    # plot related
    'plot_trjs': False,
    'spacing': 2.0,
    'accelerate_storing': True,
    'seed': 42,
    'epoch_time': experiment_id,
    'plot_bvf': False
}

sample = ap.Sample(
    parameters,
    n=saltelli_n,
    method='saltelli',
    calc_second_order=False,
    randomize=False
)
exp = ap.Experiment(CADAModel, sample, iterations=1, record=False)
results = exp.run(n_jobs=-1, verbose=10)
results.save(exp_name='E', exp_id=experiment_id, path='Experiments', display=True)
print("model complete")


## Index calculation 
Index calculation is done in R. After indices are computed and saved to disk, they are merged with agentpy output dictionary (holding the sample values) to allow computation Sobol Sensitivity analysis.

In [ ]:
! Rscript ./model_ouput_to_indices.r E_{experiment_id}

Load data and iterate over samples. For each sample split indices by motivational state and compute mean indices per motivational state. Add mean indices of all samples to agentpy results dictionary and save to file. 

In [ ]:
# Load indices
indices = pandas.read_csv("Experiments/E_" + str(experiment_id) + "/indices/indices_no_resample.csv")
# Load results dictionary
results = ap.DataDict.load(exp_name='E',  exp_id=str(experiment_id), path='Experiments', display=True)

all_reporters = pandas.DataFrame()
# iterate over samples
for s_id in range(0,len(sample)):
    sample_indices = indices.loc[indices["sampleID"] == s_id]
    # split by motivational state and add prefix to indices
    searching_indices = sample_indices.loc[sample_indices["behavior"] == "searching"]
    searching_indices = searching_indices.add_prefix('searching_')
    homing_indices = sample_indices.loc[sample_indices["behavior"] == "homing"]
    homing_indices = homing_indices.add_prefix('homing_')
    known_feeder_indices = sample_indices.loc[sample_indices["behavior"] == "known-feeder"]
    known_feeder_indices = known_feeder_indices.add_prefix('known_feeder_')
    # compute mean indices
    sim = searching_indices.mean(numeric_only=True)
    him = homing_indices.mean(numeric_only=True)
    kim = known_feeder_indices.mean(numeric_only=True)
    # add to global index DataFrame
    reporters = pandas.concat([sim, him, kim])
    all_reporters = pandas.concat([all_reporters, reporters], axis=1, ignore_index=True)
# merge results dictionary and computed mean indices and save to file 
results.reporters = pandas.concat([results.reporters.T, all_reporters]).T
results.save(exp_name="SA_", exp_id=str(experiment_id) + "_SA", path='Experiments', display=True)

Alternatively data can be loaded from file, if above cell has been run before using the following code:

```
# load data
results = ap.DataDict.load(exp_name="SA_", exp_id=<put_id_as_string_here> + "_SA", path='Experiments', display=True)
```

## Visualization
Define functions for visualizing data in three types of plots:

1. Stacked bar plots to show relative contribution of parameter to variance in model output
2. Bar plot with confidence interval for first-order and total effect indices for each parameter regarding each model output
3. Average model output values over parameter variations.

In [ ]:
def plot_stacked_bar_sobol(results, labels, motivation):
    """ Transform data and create stacked bar plot. """
    # get sobol indices by reporter
    si_list = results.sensitivity.sobol.groupby(by='reporter')
    si_conf_list = results.sensitivity.sobol_conf.groupby(by='reporter')
    
    # transform indice data
    df_s1 = pandas.DataFrame({})
    df_st = pandas.DataFrame({})
    for (key, si), (_, err), l in zip(si_list, si_conf_list, labels):
        si = si.droplevel('reporter')
        err = err.droplevel('reporter')
        s1_col = si["S1"]
        s1_col.name = l
        df_s1 = pandas.concat([df_s1, s1_col],axis=1)
        st_col = si["ST"]
        st_col.name = l
        df_st = pandas.concat([df_st, st_col],axis=1)
    
    # plot as stacked bars
    plot_stacked_bar_indices(df_s1, "First-order index", 'sensitivity_results/bars/s1_' + motivation +'.pdf', "S1 - " + motivation)
    plot_stacked_bar_indices(df_st, "Total effect index", 'sensitivity_results/bars/st_' + motivation +'.pdf', "ST - " + motivation)

def plot_stacked_bar_indices(indices, xlabel, outpath, title):
    """ Stacked bar plot of Sobol sensitivity indices (S1 and ST). """
    # rename parameter labels
    transformed = indices.T.rename(columns={'speed_impact': 'sand speed', 'veg_speed_impact': 'shrub speed', 'kphi_impact': '$k_ϕ$', 'sigma_impact': '$σ^2$', 'context': 'shrub presence'})

    # plot data
    sns.set()
    fig, axs = plt.subplots(1, 1, figsize=(10, 6))
    transformed.plot.barh(stacked=True, ax=axs, fontsize=25)
    
    # styling
    axs.set_xlabel(xlabel, fontsize=25)
    axs.set_xlim(left=0)
    axs.legend(loc='best', fontsize=25)
#     axs.set_title(title, fontsize=25, y=1.0, pad=50)

    plt.legend(bbox_to_anchor=(0, 1.02, 1, 0.2), loc="lower left", mode="expand", borderaxespad=0, ncol=3, fontsize=18)
    plt.tight_layout()
    
    # save figure and display
    plt.savefig(outpath)
    plt.show()
    plt.close()

def plot_sobol_classic(results, labels, motivation):
    """ Bar plot of Sobol sensitivity indices (S1 and ST) with confidence intervals as error bars. """
    # get sobol indices by reporter
    si_list = results.sensitivity.sobol.groupby(by='reporter')
    si_conf_list = results.sensitivity.sobol_conf.groupby(by='reporter')

    # create figure
    sns.set()
    fig, axs = plt.subplots(1, 5, figsize=(16, 10), sharey=True)

    # iterate over model outputs and plot bars for sobol indices and confidence intervals 
    for (key, si), (_, err), ax, l in zip(si_list, si_conf_list, axs, labels):
        si = si.droplevel('reporter')
        err = err.droplevel('reporter')
        si.plot.bar(yerr=err, title=l, ax=ax, capsize = 3)
        labels = ['sand speed', 'shrub speed', '$k_ϕ$', '$σ^2$', 'shrub presence']
        ax.set_xticklabels(labels)
    
    # styling
    plt.suptitle("Sobol indices for: " + motivation, fontsize=20)
    plt.tight_layout()
    plt.savefig('sensitivity_results/bars/classic_bars_'+ motivation +'.pdf')
    plt.show()
    plt.close()

# 
def plot_sensitivity(results, arr, labels, motivation):
    """ Show average simulation results for different parameter values. """

    # get data and labels
    data = results.arrange_reporters().astype('float')
    params = results.parameters.sample.keys()
    param_labels = ['sand speed','shrub speed','$k_ϕ$','$σ^2$','shrub presence']
    
    #iterate over parameters and plot into subplot
    for x, labx in zip(arr, labels):
        sns.set()
        fig, axs = plt.subplots(1, 5, figsize=(16, 8), sharey=True, sharex=True)
        for ax, y, ylab in zip(axs, params, param_labels):
            sns.regplot(x=y, y=x, data=data, ax=ax, ci=99, x_bins=15, label=ylab, fit_reg=False)
            # subplot styling/labels
            ax.set_ylabel(labx)
            ax.set_xlabel("multiplication factor value")
            ax.legend(loc='best')
            ax.set_title(ylab)
        # figure styling/labels
        plt.suptitle("Average " + labx + " for variation in parameters during " + motivation, fontsize=20)
        plt.tight_layout()
        
        # save to file and display
        plt.savefig('sensitivity_results/lines/' + x +'.pdf')
        plt.show()
        plt.close()



In [ ]:
# define labels for plots
labels_box_reduced = ["EMax", "Mean DC", "Mean speed", "Rel. shrub time", "Straightness"]
labels_box_reduced_units = ["Maximum Expected Displacement", "Mean Directional Change in °/s", "Mean speed in m/s", "Relative Time spent in Shrubs", "Straightness"]

### Initial search indices
First compute initial search indices and produce plots:

In [ ]:
# calulcate sobol indices for inital search
search_sob_results = results.calc_sobol(reporters=['searching_Emax', 'searching_mean_speed',  'searching_straightness', 'searching_directional_change_mean', 'searching_relative_vegetation_time'])

In [ ]:
# plot stacked bars (inital search)
plot_stacked_bar_sobol(search_sob_results, labels_box_reduced, "Initial search")

In [ ]:
# plot bars with confidence interval (inital search)
plot_sobol_classic(search_sob_results, labels_box_reduced, "Initial search")

In [ ]:
# plot output variance by individual parameter variance (inital search)
plot_sensitivity(search_sob_results, ['searching_Emax', 'searching_directional_change_mean', 'searching_mean_speed', 'searching_relative_vegetation_time', 'searching_straightness'], labels_box_reduced_units, "initial search")

### Oriented search indices
Next compute oriented search indices and produce plots:

In [ ]:
# calulcate sobol indices for oriented search
feeder_sob_results = results.calc_sobol(reporters=['known_feeder_Emax', 'known_feeder_mean_speed',  'known_feeder_straightness', 'known_feeder_directional_change_mean', 'known_feeder_relative_vegetation_time'])

In [ ]:
# plot stacked bars (oriented search)
plot_stacked_bar_sobol(feeder_sob_results, labels_box_reduced, "Oriented search")

In [ ]:
# plot bars with confidence interval (oriented search)
plot_sobol_classic(feeder_sob_results, labels_box_reduced, "Oriented search")

In [ ]:
# plot output variance by individual parameter variance
plot_sensitivity(feeder_sob_results, ['known_feeder_Emax', 'known_feeder_directional_change_mean', 'known_feeder_mean_speed', 'known_feeder_relative_vegetation_time', 'known_feeder_straightness'], labels_box_reduced_units, "oriented search")

### Homing indices
Finally compute homing indices and produce plots:

In [ ]:
# calulcate sobol indices for homing
homing_sob_results = results.calc_sobol(reporters=['homing_Emax', 'homing_mean_speed', 'homing_straightness', 'homing_directional_change_mean', 'homing_relative_vegetation_time'])

In [ ]:
# plot stacked bars (homing)
plot_stacked_bar_sobol(homing_sob_results, labels_box_reduced, "Homing")

In [ ]:
# plot bars with confidence interval (homing)
plot_sobol_classic(homing_sob_results, labels_box_reduced, "Homing")

In [ ]:
# plot output variance by individual parameter variance (homing)
plot_sensitivity(homing_sob_results, ['homing_Emax', 'homing_directional_change_mean', 'homing_mean_speed', 'homing_relative_vegetation_time', 'homing_straightness'], labels_box_reduced_units, "homing")